## 0. Libraries

In [ ]:
import pandas as pd
# Import train_test_split function and Decision tree classifier
from sklearn.model_selection import train_test_split
# Import the StandardScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline



## 1. Datasets

In [ ]:

# read in data
tracks = pd.read_csv('datasets/fma-rock-vs-hiphop.csv')
echonest_metrics = pd.read_json('datasets/echonest-metrics.json', precise_float=True)
# merge the relevant on track_id col
echo_tracks = echonest_metrics.merge(tracks[['genre_top', 'track_id']], on='track_id')

echo_tracks.info()

## 2. EDA
To maintain efficiency and clarity in our models, we avoid using data points that are too similar, identifying such redundancies with a correlational matrix

In [ ]:
corr_metrics = echonest_metrics.corr()
corr_metrics.style.background_gradient()

## 3. Splitting our data
Since we found no strong correlations between our features, we can proceed to split our data into two arrays: one for features and another for labels, which is the genre of the track, and then apply preprocessing steps to optimize our model development.

In [ ]:
# features
features = echo_tracks.drop(["genre_top", "track_id"], axis=1).values

# labels
labels = echo_tracks["genre_top"].values

# train test split
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, 
                                                                            random_state=10)

## 4. Normalizing the feature data
Since we didn't find strong correlations between our features, we can use principal component analysis (PCA) to reduce the number of features by rotating the data to highlight the most variance. However, PCA can be biased by features with wider value ranges, so we'll first standardize our features to have a mean of 0 and a standard deviation of 1, ensuring a fair comparison across all features.

In [ ]:
# Scaling features (train and test)
scaler = StandardScaler()
scaled_train_features = scaler.fit_transform(train_features)
scaled_test_features = scaler.transform(test_features)

## 5. PCA
Having preprocessed our data, we're ready to apply PCA to explore dimensionality reduction. We'll use scree-plots and cumulative explained ratio plots to determine the optimal number of components. Scree-plots, which plot components against the variance they explain, help us identify where the variance explained significantly drops off, known as the 'elbow', which indicates an appropriate cutoff for the number of components to use.

In [ ]:
%matplotlib inline
# Explained variance (all features)
pca = PCA()
pca.fit(scaled_train_features)
exp_variance = pca.explained_variance_ratio_

# plotting explained variance
fig, ax = plt.subplots()
ax.bar(range(pca.n_components_), exp_variance)
ax.set_xlabel('Principal Component #')

## 6. Further visualization of PCA
Even though the scree plot doesn’t clearly show where to cut off, we can use the cumulative explained variance plot to help. By aiming to explain about 85% of the variance, we can figure out how many components to keep. This way, we can reduce our data’s complexity effectively while still capturing most of its important information.

In [ ]:
# cumulative explained variance
cum_exp_variance = np.cumsum(exp_variance)

# plotting with line at 0.85
fig, ax = plt.subplots()
ax.plot(cum_exp_variance)
ax.axhline(y=0.85, linestyle='--')

## 7. Projecting on to our features
The plot showed that 6 features explain 85% of the variance. Thus, we can use these 6 components in PCA to effectively reduce the dimensionality of our train and test data.

In [ ]:
# PCA with 6 components
pca = PCA(n_components=6, random_state=10)
train_pca = pca.fit_transform(scaled_train_features)
test_pca = pca.transform(scaled_test_features)

## 8. Train a decision tree to classify genre
Now, with the lower-dimensional PCA projection of our data, we can classify songs into genres using a decision tree. This simple, rule-based classifier uses a tree structure to make binary decisions and categorize data. Decision trees are user-friendly, interpretable, and provide a visual flowchart of how decisions are made based on training data. For example, a decision tree can classify shapes by considering attributes like the number of sides and orientation.

In [ ]:
# decision tree
tree = DecisionTreeClassifier(random_state=10)
tree.fit(train_pca, train_labels)
pred_labels_tree = tree.predict(test_pca)

## 9. Compare our decision tree to a logistic regression
While our decision tree performs well, it's important not to assume it's the best option without exploring others. Next, we'll try logistic regression, which uses the logistic function to estimate the probability that a data point belongs to a specific class. By comparing the performance metrics of both models, like the false positive and false negative rates, we can determine which algorithm better suits our data.

In [ ]:
logreg = LogisticRegression(random_state=10)
logreg.fit(train_pca, train_labels)
pred_labels_logit = logreg.predict(test_pca)

class_rep_tree = classification_report(test_labels, pred_labels_tree)
class_rep_log = classification_report(test_labels, pred_labels_logit)

print("Decision Tree: \n", class_rep_tree)
print("Logistic Regression: \n", class_rep_log)

## 10. Balance our data for greater performance
Both our models have an average precision of 87%, but there's an issue with misclassification of hip-hop songs as rock. This likely occurs because our dataset contains more rock than hip-hop songs, leading the model to favor the rock classification. To improve the model's fairness, we can adjust the weights of each class in the model's calculations, giving greater importance to the underrepresented hip-hop class to balance the influence of each genre's data points. This adjustment aims to correct the skew caused by uneven sample sizes without altering the inherent value of each class.

In [ ]:
# balanced subsetting
hop_only = echo_tracks.loc[echo_tracks['genre_top'] == 'Hip-Hop']
rock_only = echo_tracks.loc[echo_tracks['genre_top'] == 'Rock']

# rock only
rock_only = rock_only.sample(hop_only.shape[0], random_state=10)

# concat hip_hop and rock dfs
rock_hop_bal = pd.concat([rock_only, hop_only])

# feature and label creation
features = rock_hop_bal.drop(['genre_top', 'track_id'], axis=1) 
labels = rock_hop_bal['genre_top']

# set train and test set with pca balancing 
train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, random_state=10)

train_pca = pca.fit_transform(scaler.fit_transform(train_features))
test_pca = pca.transform(scaler.transform(test_features))

## 11. Does balancing our dataset improve model bias?
After balancing our dataset to address the model's bias toward the "Rock" classification, we need to test if this change retains overall classification performance while correcting the bias. While we've reduced the size of our dataset in the process, we'll proceed without further dimensionality reduction. Typically, we would apply more rigorous dimensionality reduction strategies in scenarios with very large datasets where computation time is a critical factor.

In [ ]:
# trai decision tree on balanced data 
tree = DecisionTreeClassifier(random_state=10)
tree.fit(train_pca, train_labels)
pred_labels_tree = tree.predict(test_pca)

# training logistic regression on  balanced data
logreg = LogisticRegression(random_state=10)
logreg.fit(train_pca, train_labels)
pred_labels_logit = logreg.predict(test_pca)

# comparison
print("Decision Tree: \n", classification_report(test_labels, pred_labels_tree))
print("Logistic Regression: \n", classification_report(test_labels, pred_labels_logit))

## 12. Using cross-validation to evaluate our models
Balancing our dataset has successfully mitigated the bias towards the "Rock" classification. To rigorously evaluate our models, we'll implement cross-validation (CV), which provides a thorough comparison by testing the models across multiple splits of the data. We'll prepare for CV by setting up pipelines to scale our data, apply PCA, and choose a model, either the `DecisionTree22Classifier` or `LogisticRegression`. Using the K-fold CV method, we'll divide the data into equal subsets, cycling each as a test set while training on the remainder. This approach allows us to compile results from each test to give a comprehensive performance score of our models.

In [ ]:
tree_pipe = Pipeline([("scaler", StandardScaler()), ("pca", PCA(n_components=6)), 
                      ("tree", DecisionTreeClassifier(random_state=10))])
logreg_pipe = Pipeline([("scaler", StandardScaler()), ("pca", PCA(n_components=6)), 
                        ("logreg", LogisticRegression(random_state=10))])

# 10-fold cross-validation
kf = KFold(10)

# training models with KFold cv
tree_score = cross_val_score(tree_pipe, features, labels, cv=kf)
logit_score = cross_val_score(logreg_pipe, features, labels, cv=kf)

# mean of each array o scores
print("Decision Tree:", np.mean(tree_score), "Logistic Regression:", np.mean(logit_score))